In [1]:
import numpy as np
import pandapower as pp
import pandapower.networks as pn
#import pandapower.converter
#import pandapower.estimation
#import pandapower.topology
#import pandapower.plotting
#import inspect
#import pandapower.test
#pandapower.test.run_all_tests()

In [2]:
net = pn.case14()
print(net)
net.line

This pandapower network includes the following parameter tables:
   - bus (14 elements)
   - load (11 elements)
   - gen (4 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (15 elements)
   - trafo (5 elements)
   - poly_cost (5 elements)
   - bus_geodata (14 elements)
 and the following results tables:
   - res_line (15 elements)
   - res_trafo (5 elements)
   - res_ext_grid (1 element)
   - res_load (11 elements)
   - res_shunt (1 element)
   - res_gen (4 elements)


,c_nf_per_km,df,from_bus,g_us_per_km,in_service,length_km,max_i_ka,max_loading_percent,name,parallel,r_ohm_per_km,std_type,to_bus,type,x_ohm_per_km
0,768.484773,1.0,0,0.0,True,1.0,42.339020,100.0,None,1,3.532005,None,1,ol,10.783732
1,716.088084,1.0,0,0.0,True,1.0,42.339020,100.0,None,1,9.846967,None,4,ol,40.649040
10,0.000000,1.0,8,0.0,True,1.0,27479.652235,100.0,None,1,0.000014,None,9,ol,0.000037
11,0.000000,1.0,8,0.0,True,1.0,27479.652235,100.0,None,1,0.000055,None,13,ol,0.000117
12,0.000000,1.0,9,0.0,True,1.0,27479.652235,100.0,None,1,0.000035,None,10,ol,0.000083
13,0.000000,1.0,11,0.0,True,1.0,27479.652235,100.0,None,1,0.000096,None,12,ol,0.000086
14,0.000000,1.0,12,0.0,True,1.0,27479.652235,100.0,None,1,0.000074,None,13,ol,0.000151
2,637.493051,1.0,1,0.0,True,1.0,42.339020,100.0,None,1,8.563928,None,2,ol,36.080033
3,494.857619,1.0,1,0.0,True,1.0,42.339020,100.0,None,1,10.590547,None,3,ol,32.134320
4,503.590401,1.0,1,0.0,True,1.0,42.339020,100.0,None,1,10.379138,None,4,ol,31.689630


In [7]:
#net.line[["from_bus","to_bus","length_km","r_ohm_per_km","x_ohm_per_km"]]
nbus = len(net.bus)
r = net.line["length_km"]*net.line["r_ohm_per_km"]
x = net.line["length_km"]*net.line["x_ohm_per_km"]
from_buses = net.line["from_bus"].values
to_buses = net.line["to_bus"].values

z = r.values+x.values*1j
yij = 1/z

bus_connections = np.concatenate([from_buses, to_buses])
y_connections = np.concatenate([yij,yij])
yjj = np.zeros(nbus,dtype=np.complex_)
for i in range(0,nbus):
    yjj[i] = sum(y_connections[np.where((bus_connections == i))])




In [10]:
Ybus = np.zeros((nbus,nbus),dtype=np.complex_)
for i in range(0,nbus):
    Ybus[from_buses[i]][to_buses[i]] = -yij[i]
    Ybus[to_buses[i]][from_buses[i]] = -yij[i]
    Ybus[i][i]                       = yjj[i]


In [11]:
Ybus

array([[3.30591443e-02-1.06985296e-01j, 2.74300774e-02-8.37480742e-02j,
        0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
        5.62906697e-03-2.32372219e-02j, 0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j],
       [2.74300774e-02-8.37480742e-02j, 5.22432023e-02-1.66555366e-01j,
        6.22781450e-03-2.62379322e-02j, 9.25121070e-03-2.80704435e-02j,
        9.33409968e-03-2.84989158e-02j, 0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j, 0.00000000e+00+0.000000